# Analysis of Neural Programmer

In [1]:
import copy
import itertools
import os
import pickle
import re
import string
import sys
import time
from collections import defaultdict
from random import shuffle

import autoreload
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from IPython.display import HTML, Image, clear_output, display
from scipy.spatial.distance import cosine

sys.path.append('../neural_programmer')

/scratch/pramodkm/tensorflow_gpu_python3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import notebook_utils
import data_utils
from neural_programmer import evaluate

In [3]:
%reload_ext autoreload
%autoreload 2

## Paths, parameters, etc. 

In [4]:
# Use only one GPU on the multi-GPU machine
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# WikiTableQuestions data
DATA_DIR = '../wtq_data'
PERTURBED_DATA_DIR = '../perturbed_wtq_data'

# Pretrained model
MODEL_FILE = os.path.join('..', 'pretrained_model', 'model_92500')

# Output directory to write attributions
OUT_DIR = '/scratch/pramodkm/acl18/neural_programmer/'

## Load data, build graph and restore pretrained weights

In [5]:
train_data, dev_data, test_data, utility = notebook_utils.init_data(DATA_DIR)

('Annotated examples loaded ', 14152)
('Annotated examples loaded ', 4344)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/pramodkm/tensorflow_gpu_python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-322e4283c5b9>", line 1, in <module>
    train_data, dev_data, test_data, utility = notebook_utils.init_data(DATA_DIR)
  File "../neural_programmer/notebook_utils.py", line 33, in init_data
    train_data, dev_data, test_data = dat.load()
  File "../neural_programmer/wiki_data.py", line 529, in load
    self.load_annotated_tables()
  File "../neural_programmer/wiki_data.py", line 359, in load_annotated_tables
    entry = self.pre_process_sentence(tokens, ner_tags, ner_values)
  File "../neural_programmer/wiki_data.py", line 272, in pre_process_sentence
    if (tokens[i] == "score"):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/scratch/pramodkm/tensorflow_gp

KeyboardInterrupt: 

In [ ]:
tf.reset_default_graph()
sess, graph, params = notebook_utils.build_graph(utility)

In [ ]:
sess, graph = notebook_utils.restore_model(sess, graph, params, MODEL_FILE)

In [12]:
num_correct, num_examples, correct_dict = evaluate(sess, dev_data, utility.FLAGS.batch_size, graph, 92500)

('dev set accuracy   after ', 92500, ' : ', 0.37195600942655144)
(2546, 2546)
--------


ValueError: too many values to unpack (expected 2)

In [13]:
num_correct/2831.0

NameError: name 'num_correct' is not defined

In [ ]:
num_correct

## Apply Integrated Gradients (IG) 

In [ ]:
# write attributions to this folder
attrs_outdir = os.path.join(OUT_DIR, 'attributions')
if not os.path.isdir(attrs_outdir):
    os.makedirs(attrs_outdir)

# get embedding of dummy token
embeddings = graph.params["word"].eval()
dummy_embedding = embeddings[utility.dummy_token_id, :]

# which data to use?
data = dev_data

# number of sample points for Riemann integral computation
num_points = 2000

# hard coded stuff in the code
question_attention_mask_value = -10000.0

In [ ]:
batch_size = graph.batch_size

for offset in range(0, len(data) - graph.batch_size + 1, graph.batch_size):
    feed_dict = data_utils.generate_feed_dict(data, offset, graph.batch_size, graph)

    # first run inference to get operator and column sequences, and embeddings of question words
    fetches = [graph.final_correct_list, graph.final_operation_softmax,
               graph.final_column_softmax, graph.question_words_embeddings]
    correct_list, operation_softmax, column_softmax, question_words_embeddings = sess.run(
        fetches, feed_dict)

    # compute table-specific default programs for tables in this batch
    feed_copy = feed_dict.copy()
    for t in graph.question_words_embeddings:
        feed_copy[t] = np.concatenate(
            [np.expand_dims(dummy_embedding, 0)]*batch_size, 0)

    # Ideally the following line should be uncommented, but for attributions,
    # we choose to keep this variable fixed. Note that this induces some bias
    # in the attributions as the baseline is no longer an "empty" question, but
    # an empty question where the question length is implicitly encoded in this variable
    # feed_copy[graph.batch_question_attention_mask].fill(question_attention_mask_value)

    feed_copy[graph.batch_exact_match] = np.zeros_like(
        feed_copy[graph.batch_exact_match])
    feed_copy[graph.batch_column_exact_match] = np.zeros_like(
        feed_copy[graph.batch_column_exact_match])

    fetches = [graph.final_operation_softmax, graph.final_column_softmax]

    default_operation_softmax, default_column_softmax = sess.run(
        fetches, feed_copy)

    for batch_id in range(batch_size):
        wiki_example = data[offset+batch_id]

        # get operator indices
        op_indices = np.argmax(operation_softmax[batch_id, :, :], axis=1)
        col_indices = np.argmax(column_softmax[batch_id, :, :], axis=1)

        op_list = notebook_utils.softmax_to_names(
            operation_softmax[batch_id, :, :], utility.operations_set)
        col_list = notebook_utils.softmax_to_names(
            column_softmax[batch_id, :, :], notebook_utils.get_column_names(wiki_example))

        default_op_list = notebook_utils.softmax_to_names(
            default_operation_softmax[batch_id, :, :], utility.operations_set)
        default_col_list = notebook_utils.softmax_to_names(
            default_column_softmax[batch_id, :, :], notebook_utils.get_column_names(wiki_example))

        print([notebook_utils.rename(w) for w in op_list])
        print(col_list)

        # Sample points along the integral path and collect them as one batch
        scaled_feed = feed_dict.copy()
        for key in list(scaled_feed.keys()):
            value = feed_dict[key]
            if key.shape[0] == batch_size:  # this is a hack
                scaled_feed[key] = [value[batch_id] for i in range(batch_size)]
        scaled_feed[graph.op_ids] = op_indices
        scaled_feed[graph.col_ids] = col_indices

        num_examples = batch_size * int(num_points/float(batch_size))
        scale = 1.0/num_examples

        batch_op_attribution = np.zeros(
            [graph.max_passes, graph.question_length+2], dtype=np.float32)
        batch_col_attribution = np.zeros(
            [graph.max_passes, graph.question_length+2], dtype=np.float32)

        attr_op_softmax = []
        attr_col_softmax = []

        actual_num_numeric_cols = len(wiki_example.original_nc_names)
        actual_num_word_cols = len(wiki_example.original_wc_names)

        exact_match = wiki_example.exact_match
        exact_column_match = wiki_example.exact_column_match

        batch_question_embeddings = np.array(question_words_embeddings)[
            :, batch_id, :]  # shape: 62 x 256

        # split up set of points into batch_size'd batches
        for k in range(0, num_examples, batch_size):
            print('k:', k)
            # scale question words to points between dummy_embedding and actual embedding
            qw_jump = [None]*graph.question_length
            for i, t in enumerate(graph.question_words_embeddings):
                qw_jump[i] = scale * \
                    (batch_question_embeddings[i] - dummy_embedding)
                scaled_feed[t] = [dummy_embedding + j*qw_jump[i]
                                  for j in range(k, k+batch_size)]

            # scale batch_exact_match
            scaled_exact_match = []
            scaled_column_exact_match = []

            exact_match_jump = [None]*(graph.num_cols + graph.num_word_cols)
            exact_column_match_jump = [None] * \
                (graph.num_cols + graph.num_word_cols)
            for i in range(graph.num_cols):
                if i < actual_num_numeric_cols:  # do not scale dummy columns
                    scaled_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_match[i]) for j in range(k, k+batch_size)], 1))
                    exact_match_jump[i] = scale*np.array(exact_match[i])
                    scaled_column_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_column_match[i]) for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[i] = scale * \
                        np.array(exact_column_match[i])
                else:
                    scaled_exact_match.append(np.expand_dims(
                        [exact_match[i] for j in range(k, k+batch_size)], 1))
                    exact_match_jump[i] = 0
                    scaled_column_exact_match.append(np.expand_dims(
                        [exact_column_match[i] for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[i] = 0

            for i in range(graph.num_word_cols):
                if i < actual_num_word_cols:  # do not scale dummy column names
                    scaled_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_match[graph.num_cols+i]) for j in range(k, k+batch_size)], 1))
                    exact_match_jump[graph.num_cols + i] = scale * \
                        np.array(exact_match[graph.num_cols+i])
                    scaled_column_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_column_match[graph.num_cols + i]) for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[graph.num_cols + i] = scale * \
                        np.array(exact_column_match[graph.num_cols + i])
                else:
                    scaled_exact_match.append(np.expand_dims(
                        [exact_match[graph.num_cols+i] for j in range(k, k+batch_size)], 1))
                    exact_match_jump[graph.num_cols + i] = 0
                    scaled_column_exact_match.append(np.expand_dims(
                        [exact_column_match[graph.num_cols + i] for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[graph.num_cols + i] = 0

            scaled_feed[graph.batch_exact_match] = np.concatenate(
                scaled_exact_match, 1)  # shape 20 x 40 x 100
            scaled_feed[graph.batch_column_exact_match] = np.concatenate(
                scaled_column_exact_match, 1)  # shape 20 x 40

            # compute gradients
            fetches = [graph.final_operation_softmax, graph.final_column_softmax, graph.operator_gradients,
                       graph.column_gradients]
            temp_op_softmax, temp_col_softmax, operator_gradients, column_gradients = sess.run(
                fetches, scaled_feed)  # operator gradient shape: 4 x 62 x 20 x 256

            attr_op_softmax.append(temp_op_softmax)
            attr_col_softmax.append(temp_col_softmax)

            # compute attributions
            for stage in range(graph.max_passes):
                n = int(len(operator_gradients)/graph.max_passes)
                temp = [np.sum(operator_gradients[n*stage][i]*qw_jump[i], axis=(0, 1))
                        for i in range(graph.question_length)]
                temp += [np.sum([operator_gradients[n*stage+1][0][:, i, :]*exact_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                temp += [np.sum([operator_gradients[n*stage+2][0][:, i]*exact_column_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                batch_op_attribution[stage, :] += temp

            for stage in range(graph.max_passes):
                n = int(len(column_gradients)/graph.max_passes)
                temp = [np.sum(column_gradients[n*stage][i]*qw_jump[i], axis=(0, 1))
                        for i in range(graph.question_length)]
                temp += [np.sum([column_gradients[n*stage+1][0][:, i, :]*exact_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                temp += [np.sum([column_gradients[n*stage+2][0][:, i]*exact_column_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                batch_col_attribution[stage, :] += temp

        # sanity check to make sure the integral summation adds up to function difference
        attr_op_softmax = np.concatenate(attr_op_softmax, axis=0)
        attr_col_softmax = np.concatenate(attr_col_softmax, axis=0)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_op_attribution[stage, :])
            input_fn_value = operation_softmax[batch_id,
                                               stage, op_indices[stage]]
            baseline_fn_value = attr_op_softmax[0, stage, op_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('OP', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=',
                  input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs-rhs)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_col_attribution[stage, :])
            input_fn_value = column_softmax[batch_id,
                                            stage, col_indices[stage]]
            baseline_fn_value = attr_col_softmax[0, stage, col_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('COL', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=',
                  input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs-rhs)

        op_attributions = [None]*graph.max_passes
        question_begin = np.nonzero(
            wiki_example.question_attention_mask)[0].shape[0]

        attributions_matrix = np.zeros(
            [graph.question_length - question_begin + 2, 2 * graph.max_passes])
        row_labels = []  # question words, tm, cm
        col_labels = []  # operator and column selections
        col_label_softmaxes = []  # softmaxes of the selections

        for ix in range(question_begin, graph.question_length):
            word = utility.reverse_word_ids[wiki_example.question[ix]]
            if word == utility.unk_token:
                word = word + '-' + [str(w) for w in wiki_example.string_question if w !=
                                     wiki_example.question_number and w != wiki_example.question_number_1][ix - question_begin]
            word = notebook_utils.rename(word)
            row_labels.append(word)
        row_labels.extend(['tm', 'cm'])

        for stage in range(graph.max_passes):
            col_labels.append(notebook_utils.rename(
                op_list[stage]) + ' (' + notebook_utils.rename(default_op_list[stage]) + ')')
            col_labels.append(notebook_utils.rename(
                col_list[stage]) + ' (' + notebook_utils.rename(default_col_list[stage]) + ')')

            col_label_softmaxes.append(str(operation_softmax[batch_id, stage, op_indices[stage]]) + ' (' + str(
                default_operation_softmax[batch_id, stage, op_indices[stage]]) + ')')
            col_label_softmaxes.append(str(column_softmax[batch_id, stage, col_indices[stage]]) + ' (' + str(
                default_column_softmax[batch_id, stage, col_indices[stage]]) + ')')

            attributions_matrix[:, 2 * stage] = batch_op_attribution[stage, question_begin:]
            attributions_matrix[:, 2 * stage +
                                1] = batch_col_attribution[stage, question_begin:]

        question_string = ' '.join([notebook_utils.rename(str(w))
                                    for w in wiki_example.string_question])

        # save operator and column selections to file
        with tf.gfile.GFile(os.path.join(attrs_outdir, wiki_example.question_id + '_labels.tsv'), 'w') as outf:
            outf.write(question_string)
            outf.write('\n')
            outf.write(str(correct_list[batch_id] == 1.0))
            outf.write('\n')
            outf.write('\t'.join(row_labels) + '\n')
            outf.write('\t'.join(col_labels) + '\n')
            outf.write('\t'.join(col_label_softmaxes) + '\n')

        # save attributions to file
        np.savetxt(os.path.join(
            attrs_outdir, wiki_example.question_id + '_attrs.txt'), attributions_matrix)


## Create HTML with visualizations

## Apply Integrated Gradients on table-specific default programs

In [ ]:
# write attributions to this file
attrs_outdir = os.path.join(OUT_DIR, 'attributions_default_programs')
if not os.path.isdir(attrs_outdir):
    os.makedirs(attrs_outdir)

# get embedding of dummy token
embeddings = graph.params["word"].eval()
dummy_embedding = embeddings[utility.dummy_token_id, :]

# which data to use?
data = dev_data

# number of sample points for Riemann integral computation
num_points = 2000

# hard coded stuff in the code
question_attention_mask_value = -10000.0

In [ ]:
# collect all unique tables
unique_tables = {}
for wiki_example in data:
    if not wiki_example.table_key in unique_tables:
        wiki_example.exact_column_match = np.zeros_like(
            wiki_example.exact_column_match).tolist()
        wiki_example.exact_match = np.zeros_like(
            wiki_example.exact_match).tolist()
        wiki_example.question = [
            utility.dummy_token_id] * graph.question_length
        wiki_example.question_attention_mask = (question_attention_mask_value * \
            np.ones_like(wiki_example.question_attention_mask)).tolist()
        unique_tables[wiki_example.table_key] = wiki_example
data = list(unique_tables.values())

In [ ]:
for offset in range(0, len(data) - graph.batch_size + 1, batch_size):

    feed_dict = data_utils.generate_feed_dict(data, offset, batch_size, graph)
    fetches = [graph.final_correct_list, graph.final_operation_softmax,
               graph.final_column_softmax, graph.column_hidden_vectors, graph.word_column_hidden_vectors]
    correct_list, operation_softmax, column_softmax, column_hidden_vectors, word_column_hidden_vectors = sess.run(
        fetches, feed_dict)

    # compute global default program
    feed_copy = feed_dict.copy()
    feed_copy[graph.column_hidden_vectors] = np.zeros(
        graph.column_hidden_vectors.get_shape().as_list())
    feed_copy[graph.word_column_hidden_vectors] = np.zeros(
        graph.word_column_hidden_vectors.get_shape().as_list())
    default_operation_softmax, default_column_softmax = sess.run([graph.final_operation_softmax, graph.final_column_softmax], feed_copy)

    for batch_id in range(batch_size):
        wiki_example = data[offset + batch_id]

        # get op indices
        op_indices = np.argmax(operation_softmax[batch_id, :, :], axis=1)
        col_indices = np.argmax(column_softmax[batch_id, :, :], axis=1)

        op_list = notebook_utils.softmax_to_names(
            operation_softmax[batch_id, :, :], utility.operations_set)
        col_list = notebook_utils.softmax_to_names(
            column_softmax[batch_id, :, :], notebook_utils.get_column_names(wiki_example))

        print([notebook_utils.rename(w) for w in op_list])
        print(col_list)

        # generate scaled feed
        scaled_feed = feed_dict.copy()
        for key in list(scaled_feed.keys()):
            value = feed_dict[key]
            if key.shape[0] == batch_size: # this is a hack
                scaled_feed[key] = [value[batch_id] for i in range(batch_size)]
        scaled_feed[graph.op_ids] = op_indices
        scaled_feed[graph.col_ids] = col_indices

        num_examples = batch_size * int(num_points/float(batch_size))
        scale = 1.0 / num_examples

        batch_op_attribution = np.zeros(
            [graph.max_passes, graph.num_cols + graph.num_word_cols], dtype=np.float32)
        batch_col_attribution = np.zeros(
            [graph.max_passes, graph.num_cols + graph.num_word_cols], dtype=np.float32)
        
        attr_op_softmax = []
        attr_col_softmax = []

        actual_num_numeric_cols = len(wiki_example.original_nc_names)
        actual_num_word_cols = len(wiki_example.original_wc_names)
        numeric_column_name_jump = [None] * graph.num_cols
        word_column_name_jump = [None] * graph.num_word_cols
        for k in range(0, num_examples, batch_size):
            print('k:', k)
            scaled_numeric_column_names = []
            scaled_word_column_names = []

            for i in range(graph.num_cols):
                if i < actual_num_numeric_cols:  # do not scale dummy column
                    scaled_numeric_column_names.append(np.expand_dims(
                        [j * scale * np.array(column_hidden_vectors[batch_id, i, :]) for j in range(k, k + batch_size)], 1))
                    numeric_column_name_jump[i] = scale * \
                        np.array(column_hidden_vectors[batch_id, i, :])
                else:
                    scaled_numeric_column_names.append(np.expand_dims([np.array(
                        column_hidden_vectors[batch_id, i, :]) for j in range(k, k + batch_size)], 1))
                    numeric_column_name_jump[i] = 0

            for i in range(graph.num_word_cols):
                if i < actual_num_word_cols:  # do not scale dummy column names
                    scaled_word_column_names.append(np.expand_dims(
                        [j * scale * np.array(word_column_hidden_vectors[batch_id, i, :]) for j in range(k, k + batch_size)], 1))
                    word_column_name_jump[i] = scale * \
                        np.array(word_column_hidden_vectors[batch_id, i, :])
                else:
                    scaled_word_column_names.append(np.expand_dims([np.array(
                        word_column_hidden_vectors[batch_id, i, :]) for j in range(k, k + batch_size)], 1))
                    word_column_name_jump[i] = 0

            scaled_feed[graph.column_hidden_vectors] = np.concatenate(
                scaled_numeric_column_names, 1)  # shape 20 x 40 x 100
            scaled_feed[graph.word_column_hidden_vectors] = np.concatenate(
                scaled_word_column_names, 1)  # shape 20 x 40

            # compute gradients
            fetches = [graph.final_operation_softmax, graph.final_column_softmax,
                       graph.operator_gradients_default_program, graph.column_gradients_default_program]
            temp_op_softmax, temp_col_softmax, operator_gradients, column_gradients = sess.run(
                fetches, scaled_feed)  # operator gradient shape: 4 x 62 x 20 x 256

            attr_op_softmax.append(temp_op_softmax)
            attr_col_softmax.append(temp_col_softmax)

            # compute attributions
            for stage in range(graph.max_passes):
                n = int(len(operator_gradients) / graph.max_passes)
                temp = [np.sum(operator_gradients[n * stage][0][:, i, :] *
                               numeric_column_name_jump[i]) for i in range(graph.num_cols)]
                temp += [np.sum(operator_gradients[n * stage + 1][0][:, i, :] *
                                word_column_name_jump[i]) for i in range(graph.num_word_cols)]
                batch_op_attribution[stage, :] += temp

            for stage in range(graph.max_passes):
                n = int(len(column_gradients) / graph.max_passes)
                temp = [np.sum(column_gradients[n * stage][0][:, i, :] *
                               numeric_column_name_jump[i]) for i in range(graph.num_cols)]
                temp += [np.sum(column_gradients[n * stage + 1][0][:, i, :] *
                                word_column_name_jump[i]) for i in range(graph.num_word_cols)]
                batch_col_attribution[stage, :] += temp

        # sanity check
        attr_op_softmax = np.concatenate(attr_op_softmax, axis=0)
        attr_col_softmax = np.concatenate(attr_col_softmax, axis=0)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_op_attribution[stage, :])
            input_fn_value = operation_softmax[batch_id,
                                               stage, op_indices[stage]]
            baseline_fn_value = attr_op_softmax[0, stage, op_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('OP', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=', input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs - rhs)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_col_attribution[stage, :])
            input_fn_value = column_softmax[batch_id,
                                            stage, col_indices[stage]]
            baseline_fn_value = attr_col_softmax[0, stage, col_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('COL', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=', input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs - rhs)

        op_attributions = [None]*graph.max_passes

        attributions_matrix = np.zeros(
            [actual_num_numeric_cols + actual_num_word_cols, 2 * graph.max_passes])
        
        row_labels = []  # column headers
        col_labels = []  # operator and column selections
        col_label_softmaxes = []  # softmaxes of the selections

        for i in range(actual_num_numeric_cols):
            word = utility.reverse_word_ids[wiki_example.column_ids[i][0]]
            row_labels.append(word)
            
        for i in range(actual_num_word_cols):
            word = utility.reverse_word_ids[wiki_example.word_column_ids[i][0]]
            row_labels.append(word)

        non_dummy_indices = np.arange(actual_num_numeric_cols).tolist() + (np.arange(actual_num_word_cols) + graph.num_cols).tolist()
        for stage in range(graph.max_passes):
            col_labels.append(notebook_utils.rename(
                op_list[stage]) + ' (' + notebook_utils.rename(default_op_list[stage]) + ')')
            col_labels.append(notebook_utils.rename(
                col_list[stage]) + ' (' + notebook_utils.rename(default_col_list[stage]) + ')')

            col_label_softmaxes.append(str(operation_softmax[batch_id, stage, op_indices[stage]]) + ' (' + str(
                default_operation_softmax[batch_id, stage, op_indices[stage]]) + ')')
            col_label_softmaxes.append(str(column_softmax[batch_id, stage, col_indices[stage]]) + ' (' + str(
                default_column_softmax[batch_id, stage, col_indices[stage]]) + ')')

            attributions_matrix[:, 2 * stage] = batch_op_attribution[stage, non_dummy_indices]
            attributions_matrix[:, 2 * stage +
                                1] = batch_col_attribution[stage, non_dummy_indices]

        # save operator and column selections to file
        with tf.gfile.GFile(os.path.join(attrs_outdir, notebook_utils.process_table_key(wiki_example.table_key) + '_labels.tsv'), 'w') as outf:
            outf.write('\t'.join(row_labels) + '\n')
            outf.write('\t'.join(col_labels) + '\n')
            outf.write('\t'.join(col_label_softmaxes) + '\n')

        # save attributions to file
        np.savetxt(os.path.join(
            attrs_outdir, notebook_utils.process_table_key(wiki_example.table_key) + '_attrs.txt'), attributions_matrix)

## Accuracy on perturbed tables
- Perturbed data is arranged such that unperturbed questions appear before perturbed questions. This results in words being added in the same order to the vocab (to effect in same word IDs) as in the unperturbed case.
- Since the vocabulary has more words in the perturbed case (due to some words exceeding the min cutoff), special words such as `unk_token` are assigned different IDs. We revert this by swapping word IDs appropriately. The goal being that the word IDs of words in perturbed case should be the same as word IDs of words in the unperturbed case. This is done in `notebook_utils.init_data()` using the argument `preserve_vocab` (default value is `False`)

In [14]:
perturbed_train_data, perturbed_dev_data, perturbed_test_data, perturbed_utility = notebook_utils.init_data(PERTURBED_DATA_DIR, preserve_vocab=True)

('Annotated examples loaded ', 58141)
('Annotated examples loaded ', 4344)
('entry match token: ', 9169, 9169)
('entry match token: ', 9170, 9170)
hardcoded ids for special words
9133 9134 9135 9136
('# train examples ', 43835)
('# dev examples ', 10994)
('# test examples ', 3913)


In [15]:
perturbed_correct, perturbed_num_examples, perturbed_correct_dict = evaluate(sess, perturbed_dev_data, perturbed_utility.FLAGS.batch_size, graph, 92500)

('dev set accuracy   after ', 92500, ' : ', 0.2556773688332028)
(2554, 10994)
--------


In [17]:
perturbed_correct/2831

0.23066054397739313

## Question concatenation attacks

In [ ]:
ATTACK_PHRASES = [
    'in not a lot of words',
    'in this chart',
    'among these rows listed',
    'if its all the same',
    'above all',
    'at the moment'
]    

In [ ]:
num_correct_list = []
for phrase in ATTACK_PHRASES:
    prefix_correct, _ = notebook_utils.evaluate_concatenation_attack(sess, dev_data, graph.batch_size, graph, 92500, utility, phrase, suffix=False)
    suffix_correct, _ = notebook_utils.evaluate_concatenation_attack(sess, dev_data, graph.batch_size, graph, 92500, utility, phrase, suffix=True)
    num_correct_list.append([prefix_correct, suffix_correct])

In [ ]:
num_correct_list

## Stop word deletion attack

In [ ]:
STOP_WORDS = [
    'how', 'tell', 'did', 'me', 'my', 'our', 'are', 'is', 'were', 'this', 'on',
    'would', 'and', 'for', 'should', 'be', 'do', 'i', 'have', 'had', 'the',
    'there', 'look', 'give', 'has', 'was', 'we', 'get', 'does', 'a', 'an', 's',
    'that', 'by', 'based', 'in', 'of', 'bring', 'with', 'to', 'from',
    'whole', 'being', 'been', 'want', 'wanted', 'as', 'can', 'see',
    'doing', 'got', 'sorted', 'draw', 'listed', 'chart', 'only'
]

stop_word_ids = set([utility.word_ids[w] for w in STOP_WORDS if w in utility.word_ids])
question_attention_mask_value = -10000

In [ ]:
stop_word_data = copy.deepcopy(dev_data)
for i, wiki_example in enumerate(stop_word_data):
    new_question = []
    new_question_mask = []
    for w in wiki_example.question:
        if w not in stop_word_ids and w != utility.dummy_token_id:
            new_question.append(w)
            new_question_mask.append(0)
                
    stop_word_data[i].question = [utility.dummy_token_id] * (graph.question_length - len(new_question)) + new_question

    stop_word_data[i].question_attention_mask = [question_attention_mask_value] * (graph.question_length - len(new_question_mask)) + new_question_mask

In [ ]:
num_correct, _ = evaluate(sess, stop_word_data, graph.batch_size, graph, 92500)